In [ ]:
from typing import Annotated
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel
import nest_asyncio
from dotenv import load_dotenv
import json

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

In [ ]:
load_dotenv(override=True)

In [ ]:
DISCORD_API_KEY = os.getenv('DISCORD_TOKEN')
print(DISCORD_API_KEY[:4])
DISCORD_CHANNEL_ID = os.getenv('DISCORD_CHANNEL_ID')
print(DISCORD_CHANNEL_ID[:4])

In [ ]:
nest_asyncio.apply()

In [ ]:
class State(BaseModel) : 
    messages : Annotated[list, add_messages]

In [ ]:
!playwright install #only run this if you have not install the playwright browser before. the playwright library is installed in your system if you have install the requrements.txt file

While executing the next few cells, you might hit a problem with the Playwright browser raising a NotImplementedError.

This should work when we move to python modules, but it can cause problems in Windows in a notebook.

If you it this error and would like to run the notebook, you need to make a small change which seems quite hacky! You need to do this AFTER installing Playwright (prior cells)

1. Right click in `.venv` in the File Explorer on the left and select "Find in folder"
2. Search for `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())`  
3. That code should be found in a line of code in a file called `kernelapp.py`
4. Comment out the entire else clause that this line is a part of - see the fragment below. Be sure to have the "pass" statement after the ImportError line.
5. Restart the kernel by pressing the "Restart" button above

```python
        if sys.platform.startswith("win") and sys.version_info >= (3, 8):
            import asyncio
 
            try:
                from asyncio import WindowsProactorEventLoopPolicy, WindowsSelectorEventLoopPolicy
            except ImportError:
                pass
                # not affected
           # else:
            #    if type(asyncio.get_event_loop_policy()) is WindowsProactorEventLoopPolicy:
                    # WindowsProactorEventLoopPolicy is not compatible with tornado 6
                    # fallback to the pre-3.8 default of Selector
                    # asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
```

In [ ]:
async_browser = create_async_playwright_browser(headless = False)
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools() #this tools contains all the tools from the browser

In [ ]:
def send_discord_message(msg : str, channel_id : str = DISCORD_CHANNEL_ID):
    """
    :param msg: the message you want to send
    :param channel_id: channel id where you want to send your message to
    :return: None
    """
    TOKEN = DISCORD_API_KEY
    headers = {
        'Authorization': f'Bot {TOKEN}',
        'Content-Type': 'application/json'
    }

    message = json.dumps({'content': msg})

    r = requests.post(
        f'https://discord.com/api/v10/channels/{channel_id}/messages',
        headers=headers,
        data=message
    )

    if r.status_code != 200:
        print(f'Failed to send message, returned status code: {r.status_code}, response: {r.text}')
    else:
        print("Message sent successfully!")

In [ ]:
discord_push = Tool(
    name = 'discord message tool',
    func = send_discord_message,
    description='use this tool to send messages to discord'
)

In [ ]:
tools = tools + [discord_push]

In [ ]:
llm = ChatOpenAI(api_key = 'ollama', base_url = "http://127.0.0.1:11434/v1", model = 'qwen3:8b' )

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chat_bot(old_state : State) ->State : 
    response_from_llm = llm_with_tools.invoke(old_state.messages)
    new_state = State(messages=[response_from_llm])
    return new_state

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
graph_builder.add_node("chatbot" , chat_bot)
graph_builder.add_node("tools", ToolNode(tools = tools))

In [ ]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot" , tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
memory = MemorySaver()

In [ ]:
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {'configurable' : {'thread_id' : "5"}}

In [ ]:
def process_ai_response(response):
    if "<think>" in response and "</think>" in response:
        # Extract everything cleanly
        before_think = response.split("<think>")[0]
        think_content = response.split("<think>")[1].split("</think>")[0]
        after_think = response.split("</think>")[1]

        # Format the think block for display
        formatted_think = f"> 💭 *{think_content.strip()}*"

        # Combine all parts
        return after_think.strip()
    else:
        return response

#only keep this if your llm is an thinking model. This code is used to remove the part inside the <think> </think> tag so that gradio can render the output 

In [ ]:
async def chat(user_input, history):
    old_state = State(messages=[{'role' : 'user', 'content' : user_input}])
    new_state = await graph.ainvoke(old_state, config=config) #invoking the llm in async mode
    result = new_state['messages'][-1].content
    return process_ai_response(result)

In [ ]:
gr.ChatInterface(chat, type = 'messages').launch()